# Vježbe 5 - drugi dio

1. Overfitting i regularizacija
    - Linearna regresija s regularizacijom
    - Logistička regresija s regularizacijom
    - Regularizacija u scikit-learn modelima
    - Early stopping (DZ)
---

In [44]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, accuracy_score
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_blobs
from sklearn.svm import SVC

*Regularization is any modiﬁcation we make to a learning algorithm that is intended to reduce its generalization error but not it's training error.*
[Goodfellow, Bengio, Courville - Deep Learning](https://www.deeplearningbook.org/)

*Ako imate dvije teorije koje predviđaju isto, odaberite jednostavniju*. ([Occamova britva](https://hr.wikipedia.org/wiki/Occamova_britva))

Ako model ima dovoljan broj parametara, nakon treniranja je moguća pojava **overfitanja**, gdje model izvrsno opisuje skup za treniranje, ali nema kvalitetnu generalizaciju, odnosno loše opisuje nove, neviđene podatke.

Jedna metoda sprječavanja overfitanja je dodavanje regularizacijskog izraza funkciji cilja:
$$ 
J(\Theta) = J_{\textrm{old}}(\Theta) + \lambda \cdot R(\Theta),
$$

gdje je $J_{\textrm{old}}$ funkcija cilja koju smo ranije promatrali, $R(\Theta)$ je regularizacijska funkcija, a $\lambda$ je regularizacijski koeficijent koji kontrolira utjecaj regularizacijske funkcije $R(\Theta)$.

## Polinomna regresija s regularizacijom

Jednostavni regularizacijski izraz s kojim smo se upoznali je $R(\theta) = \frac{1}{2}||\Theta|| = \frac{1}{2}\Theta^T\Theta$.

U problemu linearne regresije funkcija cilja je prosječno kvadratno odstupanje te izraz s regularizacijom poprima oblik
$$ 
\frac{1}{2}\sum\limits_{i=1}^m (h_{\Theta}(x^{(i)}) - y^{(i)})^2 + \frac{\lambda}{2}\Theta^T\Theta.
$$

Ovakav regularizacijski izraz je u literaturi poznat kao *weight decay*, jer tijekom učenja u nekim iterativnim algoritmima "*potiče*" približavanje parametara prema nuli, osim ako podaci ne zahtjevaju drugačije. <br>

---
Zadatak je sljedeći:
- Učitajte podatke 'poly_x.csv' i 'poly_y.csv' i grafički ih prikažite. 
- Podatke modeliramo funkcijom $h_{\Theta, \theta_0}(x) = \theta_0 + \Theta_1 x + \Theta_2 x^2 + \Theta_3 x^3 + \Theta_4 x^4 + \Theta_5 x^5$. 
- Problem polinomne regresije svodimo na proglem višestruke linearne regresije uvođenjem novih varijabli x_i čija vrijednost će biti x^i. Kreirajte pripadnu matricu dizajna. 
- Pokrenite gradijentnu metodu s regularizacijom na vašim podacima, pri tome:
    - postavite regularizacijski koeficijent na $\lambda = 0, 1, 5, 20, 100$
    - Izračunajte srednje kvadratno odstupanje na skupu za treniranje
    - Prikažite graf funkcije $h_{\Theta, \theta_0}$
    
Što možete zaključiti na temelju različitih izbora $\lambda$? Čemu odgovara slučaj $\lambda = 0$? 

Napomena: prilikom treniranja ta tri slučaja, fiksirajte broj iteracija i stopu učenja $\alpha$.

In [60]:
def gradient_method(X, y, lmbd, lr=0.1, num_iter=50000):
    print(f'Starting gradient descent with learning rate {lr}, regularization parameter {lmbd} and {num_iter} iterations')
    m, n = X.shape
    theta = np.zeros(X.shape[1]).reshape(-1, 1)
    loss = np.empty(num_iter)
    for it in range(num_iter):
        loss[it] = (0.5 / m) * np.sum(np.square(X @ theta - y))
        # Izracunajte vrijednost funkcije cilja i gradijent
    return theta, loss

In [70]:
def plot_regression(X, y, theta, s):
    # X, y podaci
    # theta parametar modela
    # s stupanj polinoma u model funkciji h_theta
    plt.scatter(X, y)
    xx = np.linspace(np.min(X), np.max(X), 1000)
    XX = np.ones([xx.shape[0],s+1])
    for i in range(s):
        XX[:,i+1] = xx**(i+1)
    yy = np.dot(XX, theta)
    plt.plot(xx, yy, 'r')
    plt.show()

#### Validacija i stupanj polinoma
Primjenom nekog algoritma za odabir optimalnih hiperparametara možemo odrediti optimalni stupanj polinoma unutar model funkcije.

Mogli bismo iskoristiti **k-fold unakrsnu validaciju**, natrenirati više modela za svaki stupanj, te odabrati onaj koji nam daje najmanju vrijednost funkcije cilja.

## Logistička regresija s regularizacijom

Regularizirana funkcija cilja u slučaju logističke regresije ima oblik
$$ 
J(\Theta, \theta_0) = \frac{1}{m}\sum\limits_{i=1}^{m}\left[-y^{(i)}\log{(h_{\Theta, \theta_0}(x^{(i)}))}-(1-y^{(i)})\log{(1-h_{\Theta, \theta_0}(x^{(i)}))}\right] + \frac{\lambda}{2m}||\Theta||^2.
$$

Zadatak je sljedeći
- Učitajte i prikažite podatke iz datoteka 'logi_x.csv' i ' logi_y.csv'. 
- Podatke separiramo u dvije klase. Kao model funkciju koristit ćemo polinom stupnja 6. Kako ulazni podaci imaju dvije varijable, $x_1$ i $x_2$, to će biti polinom $6$-og stupnja koji sadrži sve monome od $x_1$ i $x_2$, dakle  $h_{\Theta, \theta_0}(x) = \theta_0 + \Theta_1 x_1 + \Theta_2 x_2 + \Theta_3 x_1^2 + \Theta_4 x_2^2 + \Theta_5 x_1 x_2 + \cdots + \Theta_{26} x_1 x_2^5 + \Theta_{27} x_2^6$. 
- Pomoću klase sklearn.preprocessing.PolynomialFeatures napravite odgovarajuću matricu dizajna za vaše podatke
- Koristeći implementiranu gradijentnu metodu natrenirajte vaše podatke, kao i u prethodnim zadacima napravite nekoliko slučajeva za $\lambda = 0, 1, 10, 100$. 
- Prikažite podatke i pomoću funkcije $\textrm{plot_boundary}$ prikažite izračunatu granicu. 

In [33]:
def plot_boundary(X, y, theta, poly_featurizer):
    plt.scatter(X[:, 0], X[:, 1], c=y)
    ax = plt.gca()
    xlim = ax.get_xlim()
    ylim = ax.get_ylim()

    xx, yy = np.meshgrid(np.linspace(xlim[0], xlim[1], 1000),
                         np.linspace(ylim[0], ylim[1], 1000))
    
    transformed = poly_featurizer.transform(np.c_[xx.ravel(), yy.ravel()])
    Z = (np.sign(np.dot(transformed, theta)) + 1) / 2
    
    # Z = model.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)

    plt.contour(xx, yy, Z, cmap=plt.cm.Paired)
    plt.show()

In [38]:
def sigma(z):
    return (1.0 / (1.0 + np.exp(-z)))

def predict(X, theta):
    pred = sigma(np.dot(X, theta))
    return np.array([1 if p >= 0.5 else 0 for p in pred])

def loss_fn(X, y, theta, lmbd):
    m = X.shape[0]
    h = sigma(X @ theta)
    return -(1 / m) * np.sum((y * np.log(h) + (1 - y) * np.log(1 - h))) + (lmbd / (2 * m)) * np.inner(theta[1:], theta[1:])

def gradient_method_logreg(X, y, lmbd, lr=0.1, num_iter=50000): 
    m = X.shape[0]
    n = X.shape[1]
    theta =  np.zeros((n,1))
    loss = []
    j=0
    for it in range(num_iter):     
        # Izracunajte vrijednost funkcije cilja i gradijent
                    
    return theta, loss

## Ugrađena regularizacija u SVM
Prilikom rješavanja SVM-a rješavali smo problem uvjetne optimizacije
$$
\textrm{argmin}_{\Theta, \theta_0, \xi}\frac{1}{2}||\Theta||^2 + C\sum\limits_{i=1}^m \xi_i
$$
uz uvjet
$$
\text{  uz uvjet  } y^{(i)}(\Theta^T x^{(i)} + \theta_0) \geq 1 - \xi_i \textrm{ i } \xi_i \geq 0, \text{ }  i = 1, 2, \dots, m,
$$

Cilj SVM-a je bio maksimizirati geometrijsku marginu, pa u tom kontekstu $\frac{1}{2}||\Theta||^2$ ne promatramo kao regularizacijski parametar, nego *glavni* izraz koji minimiziramo, dok $C\sum\limits_{i=1}^m \xi_i$ igra ulogu regularizacije. Hiperparametar $C$ u ovom slučaju interpretiramo kao koeficijent regularizacije, dok je "snaga" regularizacije obrnuto proporcionalna njegovoj vrijednosti.

S druge strane, u sličnom (istom do na koeficijente) problemu smo rješavali problem bezuvjetne optimizacije
$$
\textrm{argmin}_{\Theta, \theta_0}\lambda||\Theta||^2 + \frac{1}{m}\sum\limits_{i=1}^m\ell_{hinge}(y^{(i)}(\Theta^T x^{(i)} + \theta_0)).
$$
U ovom slučaju $\lambda||\Theta||^2$ interpretiramo kao regularizacijski izraz, dok je glavni cilj optimizacije minimizirati prosječan hinge gubitak.

Zadatak je natrenirati SVM model s različitim regularizacijskim koeficijentima $C$.

## Early stopping
Bez korištenja neke vrste validacije teško je odrediti u kojem trenutku treniranje kreće prema overfitanju jer gubitke i metrike računamo samo na skupu za treniranje.

Uvođenjem validacijskog skupa nakon svake iteracije optimizacijskog algoritma možemo usporediti gubitak na skupu za treniranje u skupu za validaciju. 

Ukoliko gubitak na skupu za treniranju pada, a na validacijskom skupu raste ili stagnira, najbolji odabir je ili nekako modificirati optimizacijski algoritam, ili **zaustaviti treniranje**

Više o ranom zaustavljanju u domaćoj zadaći.